In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder/Nail_psoriasis/Nail_psoriasis/Resize-450_400psoriatic-nail-001__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd.jpeg
/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder/Nail_psoriasis/Nail_psoriasis/superpixels-2003252__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd.jpeg
/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder/Nail_psoriasis/Nail_psoriasis/Crop-100450_0300psoriatic-nail-037__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd.jpeg
/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder/Nail_psoriasis/Nail_psoriasis/padd-00_10003260__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd.jpeg
/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder/Nail_psoriasis/Nail_psoriasis/superpixels-503248__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDBd.jpeg
/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder/Nail_psoriasis/Na

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm



In [3]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

# Image transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  
])

# Load dataset
data_dir = "/kaggle/input/skin-di/Skin Disease Classification Dataset/New folder"
dataset = ImageFolder(data_dir, transform=transform)

# Split into train/val
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

In [4]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(dataset.classes)
)

2025-04-23 16:20:03.110343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745425203.377873      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745425203.455303      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import torch
from torch import nn
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss
optimizer = Adam(model.parameters(), lr=5e-5, weight_decay=0.05)
criterion = nn.CrossEntropyLoss()

from tqdm import tqdm

# Training loop with progress
for epoch in range(1, 5):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

    avg_loss = total_loss / len(train_loader)
    train_acc = correct / total
    print(f"✅ Epoch {epoch}: Avg Loss = {avg_loss:.4f} | Train Accuracy = {train_acc:.2%}")


✅ Epoch 1: Avg Loss = 0.9115 | Train Accuracy = 74.60%


✅ Epoch 2: Avg Loss = 0.4866 | Train Accuracy = 87.18%


✅ Epoch 3: Avg Loss = 0.6852 | Train Accuracy = 76.74%


✅ Epoch 4: Avg Loss = 1.1144 | Train Accuracy = 54.95%


✅ Epoch 5: Avg Loss = 1.4051 | Train Accuracy = 37.11%


In [7]:
from sklearn.metrics import classification_report, accuracy_score, f1_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(pixel_values=images).logits
        preds = torch.argmax(outputs, axis=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Metrics
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))

print("✅ Accuracy:", accuracy_score(all_labels, all_preds))
print("✅ F1 Score:", f1_score(all_labels, all_preds, average='macro'))


Classification Report:
                   precision    recall  f1-score   support

   Nail_psoriasis       0.45      0.26      0.33       499
          SJS-TEN       0.50      0.62      0.55       629
         Vitiligo       0.31      0.60      0.41       429
             acne       0.00      0.00      0.00       225
hyperpigmentation       0.00      0.00      0.00       127

         accuracy                           0.41      1909
        macro avg       0.25      0.29      0.26      1909
     weighted avg       0.35      0.41      0.36      1909

✅ Accuracy: 0.40544787847040337
✅ F1 Score: 0.256534401105869


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
